In [22]:
def get_accuracy(result):
    total = result['tp']+result['tn']+result['fp']+result['fn']
    if (total == 0):
        return 0
    return (result['tp']+result['tn'])/total
    
def get_precision(result):
    total= result['tp']+result['fp']
    if (total == 0):
        return 0
    return result['tp']/total

def get_recall(result):
    total= result['tp']+result['fn']
    if (total == 0):
        return 0
    return result['tp']/total
    
def get_f1(result):
    total= get_p(result)+get_r(result)
    if (total == 0):
        return 0
    return 2 * (get_p(result)*get_r(result))/total


In [40]:
def get_verification(annotation):
    score = 0
    if ('evidences' in annotation) and ('entailment_predict' in annotation):        
        for e in annotation['entailment_predict']:
            if (e['label'] == 'entailment'):
                return 1
            elif (e['label'] == 'contradiction'):
                return -1
            else:
                score = 0.5
                
    return score
        

In [58]:
import json 
import time
from random import randint
from time import sleep

#name = "eswa_english_evidences_strict_with_text_ent_pr"
#name = "covid_5w1h_dataset_170_news_english_evidences_strict_with_text_ent"
#name = "covid_5w1h_english_evidences_strict_with_text_ent"
name = "final_dataset_v1"
with open("../results_2/"+name+".json", "r") as read_file:
    data = json.load(read_file)
    rel_values = {
        'tp' : 0,
        'fp' : 0,
        'tn' : 0,
        'fn' : 0,
    }
    evi_values = {
        'tp' : 0,
        'fp' : 0,
        'tn' : 0,
        'fn' : 0,
    }
    hybrid1_values = {
        'tp' : 0,
        'fp' : 0,
        'tn' : 0,
        'fn' : 0,
    }
    hybrid2_values = {
        'tp' : 0,
        'fp' : 0,
        'tn' : 0,
        'fn' : 0,
    }
    for k in data.keys():
        item = data[k]
        status = item['VALUE']
        reliable = False
        if (item['VALUE_reliability'] == 1):
            reliable = True
        values = []
        for a in item['annotation_values']:
            values.append(get_verification(a))        
        verifiable = False
        confidence = 0
        confidence_threshold = 0
        if (len(values)>0):
            confidence = (sum(values)*100)/len(values)
        if (confidence >= confidence_threshold):
            verifiable = True
        if (status == "True"):
            if (reliable):
                rel_values['tp'] = rel_values['tp'] + 1
                hybrid2_values['tp'] = hybrid2_values['tp'] + 1
                if (verifiable):
                    hybrid1_values['tp'] = hybrid1_values['tp'] + 1
                else:
                    hybrid1_values['fn'] = hybrid1_values['fn'] + 1
            else:
                rel_values['fn'] = rel_values['fn'] + 1
                hybrid1_values['fn'] = hybrid1_values['fn'] + 1
                if (verifiable):
                    hybrid2_values['tp'] = hybrid2_values['tp'] + 1
                else:
                    hybrid2_values['fn'] = hybrid2_values['fn'] + 1
            if (verifiable):
                evi_values['tp'] = evi_values['tp'] + 1
            else:
                evi_values['fn'] = evi_values['fn'] + 1
        else:
            if (reliable):
                rel_values['fp'] = rel_values['fp'] + 1
                hybrid2_values['fp'] = hybrid2_values['fp'] + 1
                if (verifiable):
                    hybrid1_values['fp'] = hybrid1_values['fp'] + 1
                else:
                    hybrid1_values['tn'] = hybrid1_values['tn'] + 1
            else:
                rel_values['tn'] = rel_values['tn'] + 1
                hybrid1_values['tn'] = hybrid1_values['tn'] + 1
                if (verifiable):
                    hybrid2_values['fp'] = hybrid2_values['fp'] + 1
                else:
                    hybrid2_values['tn'] = hybrid2_values['tn'] + 1
            if (verifiable):
                evi_values['fp'] = evi_values['fp'] + 1
            else:
                evi_values['tn'] = evi_values['tn'] + 1            
        print("New Item",k,":","[status:",status,"][reliable:",reliable,"][verifiable:",verifiable,"][",sum(values),"/",len(values),"][",confidence,"]")
    print("Num_Items:",len(data.keys()))    
    print("Reliability =>","p:",round(get_precision(rel_values),2),"r:",round(get_recall(rel_values),2),"accuracy:",round(get_accuracy(rel_values),2),"f1:",round(get_f1(rel_values),2))
    print("Verifiable =>","p:",round(get_precision(evi_values),2),"r:",round(get_recall(evi_values),2),"accuracy:",round(get_accuracy(evi_values),2),"f1:",round(get_f1(evi_values),2))
    print("Rel+Ver =>","p:",round(get_precision(hybrid1_values),2),"r:",round(get_recall(hybrid1_values),2),"accuracy:",round(get_accuracy(hybrid1_values),2),"f1:",round(get_f1(hybrid1_values),2))
    print("Unrel+Ver =>","p:",round(get_precision(hybrid2_values),2),"r:",round(get_recall(hybrid2_values),2),"accuracy:",round(get_accuracy(hybrid2_values),2),"f1:",round(get_f1(hybrid2_values),2))
    
    

New Item 0 : [status: True ][reliable: False ][verifiable: True ][ 1.0 / 6 ][ 16.666666666666668 ]
New Item 1 : [status: True ][reliable: False ][verifiable: True ][ 2.5 / 10 ][ 25.0 ]
New Item 2 : [status: True ][reliable: True ][verifiable: True ][ 1.5 / 12 ][ 12.5 ]
New Item 3 : [status: True ][reliable: True ][verifiable: True ][ 2.5 / 13 ][ 19.23076923076923 ]
New Item 4 : [status: True ][reliable: False ][verifiable: True ][ 1.0 / 14 ][ 7.142857142857143 ]
New Item 5 : [status: True ][reliable: False ][verifiable: True ][ 1.5 / 12 ][ 12.5 ]
New Item 6 : [status: True ][reliable: True ][verifiable: True ][ 3.0 / 10 ][ 30.0 ]
New Item 7 : [status: True ][reliable: False ][verifiable: True ][ 0 / 16 ][ 0.0 ]
New Item 8 : [status: True ][reliable: True ][verifiable: True ][ 4.5 / 21 ][ 21.428571428571427 ]
New Item 9 : [status: True ][reliable: True ][verifiable: False ][ -0.5 / 7 ][ -7.142857142857143 ]
New Item 10 : [status: True ][reliable: True ][verifiable: True ][ 0.5 / 9 ][ 5.

New Item 194 : [status: Fake ][reliable: True ][verifiable: True ][ 0 / 1 ][ 0.0 ]
New Item 195 : [status: Fake ][reliable: True ][verifiable: True ][ 0 / 2 ][ 0.0 ]
New Item 196 : [status: Fake ][reliable: True ][verifiable: True ][ 0 / 0 ][ 0 ]
New Item 197 : [status: Fake ][reliable: True ][verifiable: True ][ 0 / 2 ][ 0.0 ]
New Item 198 : [status: Fake ][reliable: True ][verifiable: True ][ 0 / 2 ][ 0.0 ]
New Item 199 : [status: Fake ][reliable: True ][verifiable: True ][ 0.5 / 2 ][ 25.0 ]
New Item 200 : [status: Fake ][reliable: True ][verifiable: True ][ 0.5 / 3 ][ 16.666666666666668 ]
New Item 201 : [status: Fake ][reliable: True ][verifiable: True ][ 0 / 0 ][ 0 ]
New Item 202 : [status: Fake ][reliable: True ][verifiable: True ][ 0 / 1 ][ 0.0 ]
New Item 203 : [status: Fake ][reliable: True ][verifiable: True ][ 0.5 / 3 ][ 16.666666666666668 ]
New Item 204 : [status: Fake ][reliable: True ][verifiable: True ][ 0 / 2 ][ 0.0 ]
New Item 205 : [status: Fake ][reliable: False ][verif